In [1]:
from datetime import datetime as dt
import logging
import yaml,os
from utils import utils
import pandas as pd
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.foundation_models.extensions.langchain import (WatsonxLLM,)
from ibm_watson_machine_learning.foundation_models.utils.enums import (ModelTypes,DecodingMethods)
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams

In [ ]:
from dotenv import load_dotenv,find_dotenv

In [2]:
def load_config():
    with open('config.yaml', 'r') as file:
        config = yaml.safe_load(file)
    return config

config = load_config()

load_dotenv()

True

In [4]:
# Read the Excel file into a DataFrame
excel_file_path = "data/Email_Data_V0.1.xlsx"
sheet_names = ["Single Query","Multiple Query"]
df = pd.read_excel(excel_file_path,sheet_name=sheet_names[0])

In [9]:
df.rename(columns = {'Customer_Query_1':'cust_qry_1',
                     'Response_1':'promotion_res_1'}, 
                     inplace = True)

In [14]:
query_response = [{"customer_query":row['cust_qry_1'],
                   "nagent_response":row['promotion_res_1']
                   }
                for _,row in df.iterrows()
                ]

In [ ]:
query_response[0]

In [17]:
from langchain.docstore.document import Document
from langchain.vectorstores import FAISS

In [18]:
query_res = []
for q_r in query_response:
    query_res.append(Document(page_content = f"query:{q_r['customer_query']},response:{q_r['promotion_res_1']}", 
                                            metadata = {"file_Name":excel_file_path, 
                                                        "Sheet_Name":sheet_names[0]}))


In [ ]:
query_res

In [20]:
from langchain.embeddings import SentenceTransformerEmbeddings
embeddings = SentenceTransformerEmbeddings(model_name=config['embeddings']['model_name']) #"all-MiniLM-L6-v2"

In [21]:
query_res_vector_db = FAISS.from_documents(query_res, embeddings)

In [30]:
query_res_vector_db.save_local(config['faiss_indexstore']['vec_db_path'], "customer_email_n_response")

In [31]:
cust_email_db = FAISS.load_local(folder_path=config['faiss_indexstore']['vec_db_path'],
                                 index_name="customer_email_n_response", 
                                 embeddings = embeddings)

In [22]:
email_13 = """
Dear XYZ,

I hope this email finds you well. My name is XXXX XXXXXX, and I am writing to follow up on the status of my claim for the freebies watch promotion that was offered with my recent XYZ purchase.

On July 02, 2023, I acquired a XYZ S23 ultra as part of the promotional offer, which entitles me to receive a free XYZ Watch. As a loyal XYZ customer, I am eagerly looking forward to taking advantage of this exciting promotion.

As of today_s date, I haven_t received any updates or notifications regarding the status of my freebies watch claim. I would greatly appreciate it if you could provide me with an update on the processing and shipment status. Additionally, if there are any further steps required from my end, kindly let me know, and I_ll be sure to fulfill them promptly.

Thank you for your attention to this matter. I truly value the XYZ brand and look forward to enjoying the benefits of this promotional offer.

I eagerly anticipate your response and the arrival of my freebies watch. Please feel free to contact me at (000000) if you require any additional information.

Best regards,
XXXXXXX
"""

In [32]:
search_results = cust_email_db.similarity_search(email_13, k=1)

In [ ]:
search_results

In [ ]:
print(search_results[0].page_content.split('response:')[-1])